# Science Plan

## Objective - test hypothesis that weather events can affect the bottom currents that control the bending and rise heights of Hydrothermal plumes 

### Plot TimeSeries to look for correlations
1. Plotted time series.... compare 2010 and 2011; overlap in seasonality. Seasonality in storms is important. Compare what is happening year to year. 
2. Data is currently has a heading and angle from verticle. Want north and east components. We have a vector in 3d, projecting that down to horizontal and splitting that out into x and y components. 
3. Karen has matlab code that solves this problem. Dax will adapt and then coach josh in how to do the same. 
4. Why does this one month have more bending above 50 degrees than other months? 

## Notes:
1. I have tried to be "pythonic" in my naming conventions: https://docs.python-guide.org/writing/style/

2. I have indexed my dataframe using time 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits import mplot3d
import pandas as pd
from datetime import datetime
import hvplot.pandas
import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh')

In [ ]:
from time import sleep

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

In [ ]:
import datetime
def dateparse (date_string):
    return datetime.datetime.strptime(date_string, '%d-%m-%Y %H:%M:%S')

## Step One - Gather Data

Plume Bending Data

In [ ]:
# Test path to data using unix command 
!ls /home/jovyan/data/covis_data/

### BendData files  (*.txt = ascii format)
* 3 variables (direction, angle from vertical, julian day)
* separator = single blank space

In [ ]:
# Take quick look at data using unix command 
!echo 'direction,deg_from_vert,jday' | cat - ~jovyan/data/covis_data/BendData2010_020113_TrakFormat.txt > ~jovyan/data/covis_data/temp.txt
!sed 's/  /,/g' ~jovyan/data/covis_data/temp.txt > ~jovyan/data/covis_data/bend_data2010.csv
!head ~jovyan/data/covis_data/bend_data2010.csv

## notes:
1. For direction, units are degrees. Both 0 degrees and 360 degrees are North.  Probably Magnetic North rather than True North.
2. Angle from vertical is also in degrees.
3. The time is in a modified Julian Day format. Modified in that it is only days since January of the appropriate year AND that it likely starts at midnight not noon.
4. The BendData are a linear fit to a centerline of an isosurface  representing a plume.

In [ ]:
path = '/home/jovyan/data/covis_data/bend_data2010.csv' # use your path
df_bd2010 = pd.read_csv(path, sep=",")
df_bd2010['year'] = '2010'
df_bd2010['datetime'] = pd.to_datetime(df_bd2010.year, format='%Y') + pd.to_timedelta(df_bd2010.jday - 1, unit='d')
df_bd2010['datetime'] = df_bd2010['datetime'].dt.round('1s')
df_bd2010 = df_bd2010.set_index('datetime')
df_bd2010.drop(['jday', 'year'], axis=1,inplace=True)
df_bd2010 = df_bd2010.rename_axis(None)
df_bd2010.head(3)

In [ ]:
df_bd2010.hvplot(x = 'index', y= ['direction'], kind = 'scatter') * df_bd2010.hvplot(x = 'index', y= ['direction'])

In [ ]:
df_bd2010.hvplot(x = 'index', y= ['direction', 'deg_from_vert'])

# Data is from COVIS data collected at Grotto Vent in the MEF.
## Centerline Vertical Flow Rate
* 7 variables (Year, Month, Day, Hour, Minute, Second, Flow rate)
* separator = single blank space
## notes:
1. Time is given as a vector here.  This was the simplest conversion from the internal Matlab datenum. 
2. Flow rate is in m/s. Values should all be <1 m/s.
3. Several rows contain only the values "NaN" which is a Matlab code for Not a Number.  These represent missing data.
4. The data in this file is from a diffent set of files than the BendData so the times will not match but be offset by 20-40 minutes.  And there will be missing values (maybe more even than the NaN's represent) as the Doppler data can only be interpretated when the plume is near vertical.

In [ ]:
# Take quick look at data using unix command 
#!head ~jovyan/data/covis_data/centerline_vertical_flow_rate_2010.txt

### This requires some clean up... almost certainly there was an cleaner way to do this but using the unix command we can clean this up and create the .csv file we want. 

In [ ]:
!echo 'year month day hour minute second flowrate' | cat - ~jovyan/data/covis_data/centerline_vertical_flow_rate_2010.txt > ~jovyan/data/covis_data/temp.txt
!sed 's/ /,/g' ~jovyan/data/covis_data/temp.txt > ~jovyan/data/covis_data/temp1.txt
!sed 's/,,,,,,,,,,,,,/,/g' ~jovyan/data/covis_data/temp1.txt > ~jovyan/data/covis_data/temp2.txt
!sed 's/,,,/,/g' ~jovyan/data/covis_data/temp2.txt > ~jovyan/data/covis_data/temp3.txt
!sed 's/^,//' <~jovyan/data/covis_data/temp3.txt >~jovyan/data/covis_data/centerline_vertical_flow_rate_2010.csv
!rm ~jovyan/data/covis_data/temp*

In [ ]:
#!head ~jovyan/data/covis_data/centerline_vertical_flow_rate_2010.csv

In [ ]:
path = '/home/jovyan/data/covis_data/centerline_vertical_flow_rate_2010.csv' # use your path
df_cvfr2010 = pd.read_csv(path, sep=",", engine='python')
df_cvfr2010= df_cvfr2010.dropna()
df_cvfr2010['year']= df_cvfr2010['year'].round(0).astype(int).astype(str)
df_cvfr2010['month']= df_cvfr2010['month'].round(0).astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_cvfr2010['day']= df_cvfr2010['day'].round(0).astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_cvfr2010['hour']= df_cvfr2010['hour'].round(0).astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_cvfr2010['minute']=df_cvfr2010['minute'].round(0).astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
#df_cvfr2010['second']= df_cvfr2010['second'].round(0).astype(int).astype(str)
df_cvfr2010['datetime'] = df_cvfr2010['year'] + df_cvfr2010['month'] + df_cvfr2010['day'] +\
'T' + df_cvfr2010['hour']+ ':' + df_cvfr2010['minute']
df_cvfr2010['datetime'] = pd.to_datetime(df_cvfr2010['datetime'])
df_cvfr2010 = df_cvfr2010.set_index('datetime')
df_cvfr2010 = df_cvfr2010[['flowrate']]
df_cvfr2010 = df_cvfr2010.rename_axis(None)
df_cvfr2010.head(3)

In [ ]:
df_cvfr2010.hvplot(x = 'index', y= 'flowrate') * df_cvfr2010.hvplot(x = 'index', y= 'flowrate', kind = 'scatter')

## Ras data
* 7 variables (Year, Month, Day, Hour, Minute, Second, Temperature)
* separator = single blank space
### notes:
1. Time is given as a vector here.  This was the simplest conversion from the internal Matlab datenum. 
2. Temperature is in degrees C. Temperature is averaged at 15 minute intervals starting at 15 minutes after the hour.
3. These data are from the RAS instument's inflow temperature sensor.  The RAS is a water sampling instrument that takes samples when a user request or inflow temperature sensor exceeds a given value.  The temperature sensor is under a "hood" that keeps the ambient ocean bottom waters from mixing with the discharging diffuse fluids. I don't expect these data to match up with the COVIS data but you never know.

In [ ]:
# Take quick look at data using unix command
!rm ~jovyan/data/covis_data/rasdata_2010.csv
!echo 'year,month,day,hour,minute,second,temp' | cat - ~jovyan/data/covis_data/rasdata_2010.txt > ~jovyan/data/covis_data/temp.txt
!sed 's/   /,/g' ~jovyan/data/covis_data/temp.txt > ~jovyan/data/covis_data/temp1.txt
!sed 's/^,//' <~jovyan/data/covis_data/temp1.txt >~jovyan/data/covis_data/rasdata_2010.csv
#!head ~jovyan/data/covis_data/rasdata_2010.csv
!rm ~jovyan/data/covis_data/temp*

In [ ]:
# Note the data type has requred a slightly a parsing scheme that does not use the .round command. 
path = '/home/jovyan/data/covis_data/rasdata_2010.csv' # use your path
df_ras2010 = pd.read_csv(path, sep=",")
df_ras2010['year']= df_ras2010['year'].astype(int).astype(str)
df_ras2010['month']= df_ras2010['month'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_ras2010['day']= df_ras2010['day'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_ras2010['hour']= df_ras2010['hour'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_ras2010['minute']=df_ras2010['minute'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_ras2010['datetime'] = df_ras2010['year'] + df_ras2010['month'] + df_ras2010['day'] +\
'T' + df_ras2010['hour']+ ':' + df_ras2010['minute']
df_ras2010['datetime'] = pd.to_datetime(df_ras2010['datetime'])
df_ras2010 = df_ras2010.set_index('datetime')
df_ras2010 = df_ras2010[['temp']]
df_ras2010 = df_ras2010.rename_axis(None)
df_ras2010.head(2)

In [ ]:
df_ras2010.hvplot(x = 'index', y= 'temp')

## Weather data
* I'm not sure how useful this file will be or what timeframe it covers.  An explanation of the variable names can be found at https://www.ndbc.noaa.gov/measdes.shtml
* weather_data_for_plotting*.txt files
* 13 variables (Year, Month, Day, Hour, Minute, Seconds, Julian day, wave height, wind direction, wind speed, wind gust speed, atmospheric pressure, air temperature)
* separator = single blank space
### notes:
1. Time is given both as a vector (from original data file probably) and as a Julian day.
2. No information on other units by m/s likely for speeds.  Rest should match the NOAA information.  
3. I probably still have the original NOAA data files for these buoys. I'll see if I can match data files to the time periods; my directories seemed a little confused.

In [ ]:
# Clean up text file and add headders 
!rm ~jovyan/data/covis_data/weather_data_for_plotting_2010_C46036.csv
!echo 'year,month,day,hour,minute,seconds,jday,wv_height,wnd_dir,wnd_spd,wnd_gspd,atm_prs,air_temp' | cat - ~jovyan/data/covis_data/weather_data_for_plotting_2010_C46036.txt > ~jovyan/data/covis_data/temp.txt
!sed 's/   /,/g' ~jovyan/data/covis_data/temp.txt > ~jovyan/data/covis_data/temp1.txt
!sed 's/^,//' <~jovyan/data/covis_data/temp1.txt >~jovyan/data/covis_data/weather_data_for_plotting_2010_C46036.csv
#!head /home/jovyan/data/covis_data/weather_data_for_plotting_2010_C46036.csv
!rm ~jovyan/data/covis_data/temp*

In [ ]:
path = '/home/jovyan/data/covis_data/weather_data_for_plotting_2010_C46036.csv' # use your path
df_wd2010c46036 = pd.read_csv(path, sep=",")
df_wd2010c46036['year']= df_wd2010c46036['year'].astype(int).astype(str)
df_wd2010c46036['month']= df_wd2010c46036['month'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['day']= df_wd2010c46036['day'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['hour']= df_wd2010c46036['hour'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['minute']=df_wd2010c46036['minute'].astype(int).astype(str).str.pad(width=2, side='left', fillchar='0')
df_wd2010c46036['datetime'] = df_wd2010c46036['year'] + df_wd2010c46036['month'] + df_wd2010c46036['day'] +\
'T' + df_wd2010c46036['hour']+ ':' + df_wd2010c46036['minute']
df_wd2010c46036['datetime'] = pd.to_datetime(df_wd2010c46036['datetime'])
df_wd2010c46036 = df_wd2010c46036.set_index('datetime')
df_wd2010c46036 = df_wd2010c46036[['jday', 'wv_height', 'wnd_dir', 'wnd_spd', 'wnd_gspd', 'atm_prs', 'air_temp']]
df_wd2010c46036 = df_wd2010c46036.rename_axis(None)
df_wd2010c46036.head(2)

In [ ]:
df_wd2010c46036.hvplot(x = 'index', y= ['wv_height'])

In [ ]:
df_wd2010c46036.hvplot(x='index', y=['wv_height',  'wnd_dir', 'wnd_spd', 'wnd_gspd', 'atm_prs', 'air_temp'], 
                width=350, height=300, subplots=True, shared_axes=False, title= True, 'wd2010c46036').cols(2)

In [ ]:
WH2010CPlot = df_WD2010C.plot(kind='line',color='black', y='Wave Height')
WH2010CPlot.set_title("Wave Height 2010 at C46036")
WH2010CPlot.set_xlabel('Date Time')
WH2010CPlot.set_ylabel('Wave Height')
plt.savefig('WHCPlot2010.png')

In [ ]:
WDir2010CPlot = df_WD2010C.plot(kind='line',color='black', y='Wind Direction')
WDir2010CPlot.set_title("Wind Direction 2010 at C46036")
WDir2010CPlot.set_xlabel('Date Time')
WDir2010CPlot.set_ylabel('Wind Direction')
plt.savefig('WDirCPlot2010.png')

In [ ]:
WS2010CPlot = df_WD2010C.plot(kind='line',color='black', y='Wind Speed')
WS2010CPlot.set_title("Wind Speed 2010 at C46036")
WS2010CPlot.set_xlabel('Date Time')
WS2010CPlot.set_ylabel('Wind Speed (m/s)')
plt.savefig('WSCPlot2010.png')

In [ ]:
WGS2010CPlot = df_WD2010C.plot(kind='line',color='black', y='Wind Gust Speed')
WGS2010CPlot.set_title("Wind Gust Speed 2010 at C46036")
WGS2010CPlot.set_xlabel('Date Time')
WGS2010CPlot.set_ylabel('Wind Gust Speed (m/s)')
plt.savefig('WGCPlot2010.png')

In [ ]:
AP2010CPlot = df_WD2010C.plot(kind='line',color='black', y='Atmospheric Pressure')
AP2010CPlot.set_title("Atmospheric Pressure 2010 at C46036")
AP2010CPlot.set_xlabel('Date Time')
AP2010CPlot.set_ylabel('Atmospheric Pressure')
plt.savefig('APCPlot2010.png')

In [ ]:
AT2010CPlot = df_WD2010C.plot(kind='line',color='black', y='Air Temperature')
AT2010CPlot.set_title("Air Temperature 2010 at C46036")
AT2010CPlot.set_xlabel('Date Time')
AT2010CPlot.set_ylabel('Air Temperature')
plt.savefig('ATCPlot2010.png')

Weather Data 2011 C46036

In [ ]:
WH2011CPlot = df_WD2011C.plot(kind='line',color='black', y='Wave Height')
WH2011CPlot.set_title("Wave Height 2011 at C46036")
WH2011CPlot.set_xlabel('Date Time')
WH2011CPlot.set_ylabel('Wave Height')
plt.savefig('WHCPlot2011.png')

In [ ]:
WDir2011CPlot = df_WD2011C.plot(kind='line',color='black', y='Wind Direction')
WDir2011CPlot.set_title("Wind Direction 2011 at C46036")
WDir2011CPlot.set_xlabel('Date Time')
WDir2011CPlot.set_ylabel('Wind Direction')
plt.savefig('WDirCPlot2011.png')

In [ ]:
WS2011CPlot = df_WD2011C.plot(kind='line',color='black', y='Wind Speed')
WS2011CPlot.set_title("Wind Speed 2011 at C46036")
WS2011CPlot.set_xlabel('Date Time')
WS2011CPlot.set_ylabel('Wind Speed (m/s)')
plt.savefig('WSCPlot2011.png')

In [ ]:
WGS2011CPlot = df_WD2011C.plot(kind='line',color='black', y='Wind Gust Speed')
WGS2011CPlot.set_title("Wind Gust Speed 2011 at C46036")
WGS2011CPlot.set_xlabel('Date Time')
WGS2011CPlot.set_ylabel('Wind Gust Speed (m/s)')
plt.savefig('WGSCPlot2011.png')

In [ ]:
AP2011CPlot = df_WD2011C.plot(kind='line',color='black', y='Atmospheric Pressure')
AP2011CPlot.set_title("Atmospheric Pressure 2011 at C46036")
AP2011CPlot.set_xlabel('Date Time')
AP2011CPlot.set_ylabel('Atmospheric Pressure')
plt.savefig('APCPlot2011.png')

In [ ]:
AT2011CPlot = df_WD2011C.plot(kind='line',color='black', y='Air Temperature')
AT2011CPlot.set_title("Air Temperature 2011 at C46036")
AT2011CPlot.set_xlabel('Date Time')
AT2011CPlot.set_ylabel('Air Temperature')
plt.savefig('ATCPlot2011.png')

Weather Data 2010 Tillamook

In [ ]:
WDir2010TPlot = df_WD2010T.plot(kind='line',color='black', y='Wind Direction')
WDir2010TPlot.set_title("Wind Direction 2010 at Tillamook")
WDir2010TPlot.set_xlabel('Date Time')
WDir2010TPlot.set_ylabel('Wind Direction')
plt.savefig('WDirTPlot2010.png')

In [ ]:
WS2010TPlot = df_WD2010T.plot(kind='line',color='black', y='Wind Speed')
WS2010TPlot.set_title("Wind Speed 2010 at Tillamook")
WS2010TPlot.set_xlabel('Date Time')
WS2010TPlot.set_ylabel('Wind Speed (m/s)')
plt.savefig('WSTPlot2010.png')

In [ ]:
WGS2010TPlot = df_WD2010T.plot(kind='line',color='black', y='Wind Gust Speed')
WGS2010TPlot.set_title("Wind Gust Speed 2010 at Tillamook")
WGS2010TPlot.set_xlabel('Date Time')
WGS2010TPlot.set_ylabel('Wind Gust Speed (m/s)')
plt.savefig('WGSTPlot2010.png')

In [ ]:
AT2010TPlot = df_WD2010T.plot(kind='line',color='black', y='Air Temperature')
AT2010TPlot.set_title("Air Temperature 2010 at Tillamook")
AT2010TPlot.set_xlabel('Date Time')
AT2010TPlot.set_ylabel('Air Temperature')
plt.savefig('ATTPlot2010.png')

Weather Data 2011 Tillamook

In [ ]:
WDir2011TPlot = df_WD2011T.plot(kind='line',color='black', y='Wind Direction')
WDir2011TPlot.set_title("Wind Direction 2011 at Tillamook")
WDir2011TPlot.set_xlabel('Date Time')
WDir2011TPlot.set_ylabel('Wind Direction')
plt.savefig('WDirTPlot2011.png')

In [ ]:
WS2011TPlot = df_WD2011T.plot(kind='line',color='black', y='Wind Speed')
WS2011TPlot.set_title("Wind Speed 2011 at Tillamook")
WS2011TPlot.set_xlabel('Date Time')
WS2011TPlot.set_ylabel('Wind Speed (m/s)')
plt.savefig('WSTPlot2011.png')

In [ ]:
WGS2011TPlot = df_WD2011T.plot(kind='line',color='black', y='Wind Gust Speed')
WGS2011TPlot.set_title("Wind Gust Speed 2011 at Tillamook")
WGS2011TPlot.set_xlabel('Date Time')
WGS2011TPlot.set_ylabel('Wind Gust Speed (m/s)')
plt.savefig('WGSTPlot2011.png')

In [ ]:
AT2011TPlot = df_WD2011T.plot(kind='line',color='black', y='Air Temperature')
AT2011TPlot.set_title("Air Temperature 2011 at Tillamook")
AT2011TPlot.set_xlabel('Date Time')
AT2011TPlot.set_ylabel('Air Temperature')
plt.savefig('ATTPlot2011.png')